In [ ]:
setprecision(7100);

In [ ]:
function Wfile(A,Times,figure)
    if figure == 1
        XLSX.openxlsx("LorenzF_J.xlsx", mode="w") do xf
        sheet = xf[1]
        XLSX.rename!(sheet, "new_sheet")
        sheet["A1"] = "X"
        sheet["B1"] = "Y"
        sheet["C1"] = "Z"
        str1="A2:C"*string(Times+1)
        sheet[str1] = A
        end
    else
        XLSX.openxlsx("LorenzF_J.xlsx", mode="rw") do xf
        sheet = xf[1]
        str2="A"*string((figure-1)*Times+2)*":C"*string(figure*Times+1)
        sheet[str2] = A 
        end
    end        
end

In [ ]:
function accuracy(t)
    e = ep*exp(BigFloat(2.3025851)*(BigFloat(0.4)*t-BigFloat(20)))
    ac = (0.29803-log10(e))/0.30103;
    return max(round(Int,ac),128);
end

In [ ]:
function order()
    M=400;
    return M+1;
end

In [ ]:
function Ep(t)
    return ep*exp(BigFloat(2.3025851)*(BigFloat(0.4)*t-BigFloat(10)))
end

In [ ]:
function steps(Dfm,Co)
    h = Co*abs(Dfm)^(BigFloat(-1)/BigFloat(order()-1))
    return min(h,BigFloat(1)/BigFloat(10))
end

In [ ]:
function goonm(mor)
    sgo = BigFloat(1)
    for i in 1:mor
        sgo = sgo*BigFloat(i)
    end
    return (sgo)^(BigFloat(1)/BigFloat(mor))
end 

In [ ]:
function goon(mor,ep)
    sgo = BigFloat(1)
    for i in 1:mor
        sgo = sgo*BigFloat(i)
    end
    return (ep*sgo)^(BigFloat(1)/BigFloat(mor))
end 

In [ ]:
function JCh(k,i)
     
    k = BigFloat(k);
    i = BigFloat(i);
    s = BigFloat(1);
    if i < k/2
        i = k - i;
    end
    for j in range(k,i+1,step=-1)
        
        j1=BigFloat(j)
        s = s*j1/(j1-i)
        
    end
    
    return s
end

In [ ]:
function DJCh()
    Cki = Array{BigFloat}(undef,order()-1,order()-1);
    for k in 1:order()-1
        for j in 1:k
            Cki[k,j] = 1
        end
    end
    
    for k in 2:order()-2
        for i in 1:k-1
           Cki[k+1,i+1] = Cki[k,i] + Cki[k,i+1]
        end
    end
    return Cki
end

In [ ]:
function multi(hstep)
    S = Array{BigFloat}(undef,1,order());
    S[1] = BigFloat(1)
    
    for i in 1:order()-1
        S[i+1] = S[i]*hstep/BigFloat(i)
    end
    
    return S
end

In [ ]:
function Sumxyz(k,x,y,Cki)
    if k < 60
        s = sum(Cki[k+1,1:k+1].*reverse!(x).*y)
        return s
    else
        s = Array{BigFloat}(undef,k+1);
        Threads.@threads for i in 0:k
            s[i+1] = Cki[k+1,i+1]*x[k-i+1]*y[i+1];
        end
        return sum(s);   
    end
end 

In [ ]:
function Talor(x,y,z,Cki,Co)
    for k in 1:order()-1
        x[k+1] = c*(-x[k] + y[k])
        y[k+1] = R*x[k] - y[k] - Sumxyz(k-1,x[1:k],z[1:k],Cki)
        z[k+1] = Sumxyz(k-1,x[1:k],y[1:k],Cki) - b*z[k]
    end
    return [x,y,z],min(steps(x[end],Co),steps(y[end],Co),steps(z[end],Co))
end

In [ ]:
function Iteration(x,y,z,Times,Cki,Lorenz,Tin,Tinf)
    stl = 0;
    flag = 0;
    for st in 1:Times
        
        Co = Mo*Ep(Tin)^(BigFloat(1)/BigFloat(order()))
        setprecision(accuracy(Tin))
        
        Tay,hstep = Talor(x,y,z,Cki,Co)

        if Tin + hstep < Tinf 
            x = Tay[1]
            y = Tay[2]
            z = Tay[3]
            Dmulti = multi(hstep);
        
            x[1] = sum(Dmulti[1:order()].*x[1:order()]);
            y[1] = sum(Dmulti[1:order()].*y[1:order()]);
            z[1] = sum(Dmulti[1:order()].*z[1:order()]);
    
            Lorenz[st,:] = [x[1],y[1],z[1]]; 
            
            Tin = Tin + hstep
            stl = st
        else
            hstep = Tinf - Tin
            x = Tay[1]
            y = Tay[2]
            z = Tay[3]
            Dmulti = multi(hstep);
        
            x[1] = sum(Dmulti[1:order()].*x[1:order()]);
            y[1] = sum(Dmulti[1:order()].*y[1:order()]);
            z[1] = sum(Dmulti[1:order()].*z[1:order()]);
    
            Lorenz[st,:] = [x[1],y[1],z[1]]; 
            Tin = Tin + hstep
            stl = st
            flag = 1
            break;
        end
           
    end
    
    Times2 = stl
    return Lorenz,Tin,Times2,flag
    
end

In [ ]:
function Lmain(x,y,z,Times,Cki,Maxi,Lorenz,Tin,Tinf)
    Times2 = 0;
    for i in 1:Maxi
    
        Lorenz,Tin,Times2,flag = Iteration(x,y,z,Times,Cki,Lorenz,Tin,Tinf)
    
        x[1] = Lorenz[end,1]
        y[1] = Lorenz[end,2]
        z[1] = Lorenz[end,3]
    
        Lorenzf = convert.(Float32,Lorenz)
        Lorenztol[Times*(i-1)+1:Times*(i-1)+Times2,:] = Lorenzf[1:Times2,:]
    
        println(i,")","LTU=",convert.(Float16,Tin),": ",Lorenzf[Times2,:]," (Ep: ",convert(Float16,log10(abs(Ep(Tin)))),") (Mp: ",accuracy(Tin),")")
                
        if flag == 1 
            break;
        end
    end
    return Lorenztol,Times2
end

In [ ]:
#defining
using XLSX

Tin = BigFloat(0);
Tinf = BigFloat(1000);
Times = 250;     #attension to use steps
Maxi = 500;             #tatol time = Max * Times(down)

ep = BigFloat(10)^BigFloat(-410);
Mo = goonm(order())
#Co = goon(order()-1,ep)

c = BigFloat(10);
R = BigFloat(28) #+ BigFloat(1)/BigFloat(2);
b = BigFloat(8)/BigFloat(3);

x = Array{BigFloat}(undef,order())
y = Array{BigFloat}(undef,order())
z = Array{BigFloat}(undef,order())
x[1] = BigFloat(-15) - BigFloat(8)/BigFloat(10); 
y[1] = BigFloat(-17) - BigFloat(48)/BigFloat(100); 
z[1] = BigFloat(35) + BigFloat(64)/BigFloat(100);

Lorenz = Array{BigFloat}(undef,Times,3);
Lorenzf = Array{Float32}(undef,Times,3);
Lorenztol = Array{Float32}(undef,Times*Maxi,3);
Cki = Array{BigFloat}(undef,order()-1,order()-1);
Dmulti = Array{BigFloat}(undef,1,order());

Cki = DJCh();

In [ ]:
#Main 
Lorenztol,Times2 = Lmain(x,y,z,Times,Cki,Maxi,Lorenz,Tin,Tinf);

In [ ]:
#drawing
using Plots; 
plot(Lorenztol[1:Times*(Maxi-1)+Times2,1],Lorenztol[1:Times*(Maxi-1)+Times2,2],Lorenztol[1:Times*(Maxi-1)+Times2,3])

In [ ]:
plot(1:Times*Maxi,Lorenztol[:,1])

In [ ]:
#=output
Lorenztol[end,:]
=#

In [ ]:
#=profiling
using ProfileView
@profview Lmain(x,y,z,Times,Cki,Maxi,Lorenz);
=#

In [ ]:
function jiecheng(nn)
    ss = BigFloat(1)
    for i in 1:nn
        ss = ss*BigFloat(i)
    end
return ss
end

jiecheng(400)

In [ ]:
Co
BigFloat(10)^BigFloat(-500)

In [ ]:
hstep

In [ ]:
Tin